In [1]:
import pandas as pd 
import numpy as np
from scipy.optimize import minimize
from functools import partial
from statsmodels.formula.api import ols

In [2]:
df = pd.read_pickle('../data_sources/echp/echp_incomes.pkl')
df.head()

wave    year   age  co         Y      logY
country hid   pid                                               
2       7601  76101    1.0  1994.0  42.0  DK  0.829935 -0.186408
        8401  84101    1.0  1994.0  70.0  DK  1.082765  0.079518
        10701 107101   1.0  1994.0  22.0  DK  0.770043 -0.261310
        16501 165101   1.0  1994.0  46.0  DK  1.336161  0.289800
        19901 199101   1.0  1994.0  23.0  DK  0.821379 -0.196771

In [3]:
df = df[(df['age']>=25) & (df['age']<=85)]
df.describe()

,wave,year,age,Y,logY
count,48797.000000,48797.000000,48797.000000,48797.000000,48797.000000
mean,4.333689,1999.628092,57.594832,1.006108,-0.151534
std,2.306214,7.922599,18.381527,0.533914,0.655234
min,1.000000,1994.000000,25.000000,0.000315,-8.064229
25%,2.000000,1995.000000,40.000000,0.639110,-0.447679
50%,4.000000,1997.000000,62.000000,0.900252,-0.105081
75%,6.000000,1999.000000,73.000000,1.254588,0.226807
max,8.000000,2021.000000,85.000000,3.519351,1.258277


In [4]:
def vech(cov):
	T = cov.shape[0]
	v = []
	for i in range(T):
		for j in range(i,T):
			v.append(cov[j,i])
	return np.array(v)
def mdfunc(theta,cov,weight):
	rho = theta[0]
	sige = theta[1]
	sigv = theta[2]
	T = cov.shape[0]
	cov_true = np.zeros((T,T))
	for i in range(T):
		for j in range(T):
			cov_true[i,j] = (i==j)*sigv + (rho**(np.abs(i-j)))*sige/(1.0-rho**2)
	d = vech(cov) - vech(cov_true)
	func = d.reshape((1,len(d))) @ weight @ d.reshape((len(d),1))
	return func[0]

In [5]:
np.random.seed(seed=1234)
def get_cov(data,reps):
	T = len(data.columns)
	cov = data.cov().to_numpy()
	vcov = vech(cov)
	weight = vcov.reshape((vcov.shape[0],1))@vcov.reshape((1,vcov.shape[0]))
	weight = np.zeros((weight.shape[0],weight.shape[1]))
	for r in range(reps):
		rcov = data.sample(n=len(data),replace=True).cov().to_numpy()
		vrcov = vech(rcov)
		weight += vrcov.reshape((vrcov.shape[0],1))@vrcov.reshape((1,vrcov.shape[0]))
	weight = np.linalg.inv(weight/reps)
	return cov, weight

In [6]:
def get_pars(co):
	data = df.loc[(df['co']==co) & (df['logY'].isna()==False),:]
	data = data.reset_index()
	data = data.loc[:,['pid','wave','age','logY']]
	model = ols('logY ~ C(age)',data=data).fit()
	data.loc[:,'e'] = model.resid.to_list()
	data = data.loc[:,['pid','wave','e']]
	data = pd.pivot(data,index='pid',columns='wave')
	cov  = data.cov().to_numpy()
	weight = np.eye(vech(cov).shape[0])
	f = partial(mdfunc,cov=cov,weight=weight)
	itheta = np.array([0.9,0.02,0.02])
	opt = minimize(f,x0=itheta)
	return opt.x
	

In [7]:
countries = ['DE','DK','FR','NL','IT','SP']
table = pd.DataFrame(index=['rho','sige','sigv'],columns=countries)
for co in countries:
	table[co] = get_pars(co)
table['SE'] = table.loc[:,'DK']
table

,DE,DK,FR,NL,IT,SP,SE
rho,0.904457,0.908792,0.958514,0.945239,0.919130,0.968671,0.908792
sige,0.048186,0.016419,0.017261,0.017505,0.033790,0.015916,0.016419
sigv,0.124999,0.103823,0.187817,0.159557,0.169408,0.393699,0.103823


In [8]:
table = table.loc[:,['DE','DK','FR','IT','NL','SE','SP']]
table

,DE,DK,FR,IT,NL,SE,SP
rho,0.904457,0.908792,0.958514,0.919130,0.945239,0.908792,0.968671
sige,0.048186,0.016419,0.017261,0.033790,0.017505,0.016419,0.015916
sigv,0.124999,0.103823,0.187817,0.169408,0.159557,0.103823,0.393699


In [9]:
df = pd.read_pickle('../data_sources/psid/psid_incomes.pkl')
df.head()

,co,pid,year,age,Y,logY,wave
0,US,4001,1989,67.0,0.763534,-0.269798,1
2,US,4003,1989,38.0,0.954480,-0.046588,1
6,US,4008,1989,24.0,0.725096,-0.321451,1
15,US,4170,1989,34.0,0.836848,-0.178113,1
16,US,4172,1989,40.0,1.657952,0.505583,1


In [10]:
df = df.loc[(df['age']>=25) & (df['age']<=85),:]
df.describe()

,pid,year,age,Y,logY,wave
count,6.386300e+04,63863.000000,63863.000000,63863.000000,63863.000000,63863.000000
mean,3.513763e+06,1993.044298,45.825658,1.035352,-0.262842,5.044298
std,2.305610e+06,2.517794,15.301322,0.723412,0.906397,2.517794
min,4.001000e+03,1989.000000,25.000000,0.000030,-10.427394,1.000000
25%,1.401003e+06,1991.000000,34.000000,0.491355,-0.710588,3.000000
50%,2.679003e+06,1993.000000,42.000000,0.886871,-0.120055,5.000000
75%,5.821001e+06,1995.000000,56.000000,1.418418,0.349542,7.000000
max,6.872177e+06,1997.000000,85.000000,4.290564,1.456418,9.000000


In [11]:
pars = get_pars('US')
table.loc[:,'US'] = pars

In [12]:
table.loc['sigs',:] = table.loc['sige',:]/(1.0-table.loc['rho',:]**2)

In [13]:
table.round(3).to_latex('../tables/table_5_income_shocks.tex')

In [14]:
params = table.loc[['rho','sige'],:]
params.to_pickle('../model/params/income_shocks.pkl')